In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd

from pathlib import Path

In [3]:
train_dir = Path('E:\data\RSNA2024')

In [4]:
class CFG:
    random_seed = 42

    image_size = 256
    
    ROOT_FOLDER = train_dir / 'original'
    DEST_FOLDER = train_dir
    PNG_DIR = DEST_FOLDER / f'pngs_{image_size}'
    IMAGES_DIR = ROOT_FOLDER / 'train_images'
    TRAIN_CSV = ROOT_FOLDER / 'train.csv'
    FILES_CSV = ROOT_FOLDER / 'train_files.csv'
    TRAIN_DESC_CSV = ROOT_FOLDER / 'train_series_descriptions.csv'
    COORDS_CSV = ROOT_FOLDER / 'train_label_coordinates.csv'

### Train_df

In [5]:
train_df = pd.read_csv(CFG.TRAIN_CSV)
train_desc_df = pd.read_csv(CFG.TRAIN_DESC_CSV)

train_df.shape, train_desc_df.shape

((1975, 26), (6294, 3))

In [6]:
train_desc_df['ss_id'] = train_desc_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)

In [7]:
train_df.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [11]:
train_df.isna().sum(axis=1).sum()

572

In [8]:
cols = train_df.columns[1:]
# first = [c.split('_')[:-2] for c in cols]
# last = [c.split('_')[-2:] for c in cols]

cols = [c.split('_') for c in cols]
cols = [''.join([i[0] if len(i) > 2 else i for i in c]).upper() for c in cols]

cols = ['study_id'] + cols

cols[:5]

['study_id', 'SCSL1L2', 'SCSL2L3', 'SCSL3L4', 'SCSL4L5']

In [9]:
dict(zip(train_df.columns, cols))

{'study_id': 'study_id',
 'spinal_canal_stenosis_l1_l2': 'SCSL1L2',
 'spinal_canal_stenosis_l2_l3': 'SCSL2L3',
 'spinal_canal_stenosis_l3_l4': 'SCSL3L4',
 'spinal_canal_stenosis_l4_l5': 'SCSL4L5',
 'spinal_canal_stenosis_l5_s1': 'SCSL5S1',
 'left_neural_foraminal_narrowing_l1_l2': 'LNFNL1L2',
 'left_neural_foraminal_narrowing_l2_l3': 'LNFNL2L3',
 'left_neural_foraminal_narrowing_l3_l4': 'LNFNL3L4',
 'left_neural_foraminal_narrowing_l4_l5': 'LNFNL4L5',
 'left_neural_foraminal_narrowing_l5_s1': 'LNFNL5S1',
 'right_neural_foraminal_narrowing_l1_l2': 'RNFNL1L2',
 'right_neural_foraminal_narrowing_l2_l3': 'RNFNL2L3',
 'right_neural_foraminal_narrowing_l3_l4': 'RNFNL3L4',
 'right_neural_foraminal_narrowing_l4_l5': 'RNFNL4L5',
 'right_neural_foraminal_narrowing_l5_s1': 'RNFNL5S1',
 'left_subarticular_stenosis_l1_l2': 'LSSL1L2',
 'left_subarticular_stenosis_l2_l3': 'LSSL2L3',
 'left_subarticular_stenosis_l3_l4': 'LSSL3L4',
 'left_subarticular_stenosis_l4_l5': 'LSSL4L5',
 'left_subarticular_ste

In [10]:
train_df.rename(columns=dict(zip(train_df.columns, cols)), inplace=True)
train_df.shape

(1975, 26)

In [11]:
train_df.iloc[0]

study_id        4003253
SCSL1L2     Normal/Mild
SCSL2L3     Normal/Mild
SCSL3L4     Normal/Mild
SCSL4L5     Normal/Mild
SCSL5S1     Normal/Mild
LNFNL1L2    Normal/Mild
LNFNL2L3    Normal/Mild
LNFNL3L4    Normal/Mild
LNFNL4L5       Moderate
LNFNL5S1    Normal/Mild
RNFNL1L2    Normal/Mild
RNFNL2L3    Normal/Mild
RNFNL3L4       Moderate
RNFNL4L5       Moderate
RNFNL5S1    Normal/Mild
LSSL1L2     Normal/Mild
LSSL2L3     Normal/Mild
LSSL3L4     Normal/Mild
LSSL4L5        Moderate
LSSL5S1     Normal/Mild
RSSL1L2     Normal/Mild
RSSL2L3     Normal/Mild
RSSL3L4     Normal/Mild
RSSL4L5     Normal/Mild
RSSL5S1     Normal/Mild
Name: 0, dtype: object

In [12]:
train_df.study_id.nunique()

1975

In [13]:
vals = {'Normal/Mild': 'N', 'Moderate': 'M', 'Severe': 'S'}
vals

{'Normal/Mild': 'N', 'Moderate': 'M', 'Severe': 'S'}

In [14]:
train_df[cols[1:]] = train_df[cols[1:]].replace(vals)

In [15]:
train_df.sample(2)

,study_id,SCSL1L2,SCSL2L3,SCSL3L4,SCSL4L5,SCSL5S1,LNFNL1L2,LNFNL2L3,LNFNL3L4,LNFNL4L5,...,LSSL1L2,LSSL2L3,LSSL3L4,LSSL4L5,LSSL5S1,RSSL1L2,RSSL2L3,RSSL3L4,RSSL4L5,RSSL5S1
764,1690291272,N,N,N,N,N,N,M,M,M,...,N,N,N,N,N,N,N,N,N,N
825,1827151112,N,N,N,N,N,N,N,M,M,...,N,N,N,N,N,N,N,N,N,N


### Coordinates_df

In [16]:
coords_df = pd.read_csv(CFG.COORDS_CSV)
files_df = pd.read_csv(CFG.FILES_CSV)

coords_df.shape, files_df.shape

((48692, 7), (147218, 11))

In [17]:
coords_df.rename(columns={'instance_number': 'instance'}, inplace=True)

In [18]:
coords_df.study_id.nunique(), coords_df.condition.nunique(), coords_df.level.nunique()

(1974, 5, 5)

In [19]:
coords_df.condition.unique(), coords_df.level.unique()

(array(['Spinal Canal Stenosis', 'Right Neural Foraminal Narrowing',
        'Left Neural Foraminal Narrowing', 'Left Subarticular Stenosis',
        'Right Subarticular Stenosis'], dtype=object),
 array(['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1'], dtype=object))

In [20]:
coords_df.series_id.nunique()

6291

In [21]:
coords_df['ss_id'] = coords_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)
coords_df['instance_id'] = coords_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}_{str(row["instance"])}', axis=1)

In [22]:
train_desc_df.sample()

,study_id,series_id,series_description,ss_id
4061,2780669808,1449964301,Axial T2,2780669808_1449964301


In [23]:
coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id
16824,1505795551,2733545907,11,Left Neural Foraminal Narrowing,L3/L4,262.687023,192.366412,1505795551_2733545907,1505795551_2733545907_11
19444,1745732011,1893182379,5,Right Neural Foraminal Narrowing,L2/L3,471.899461,216.420108,1745732011_1893182379,1745732011_1893182379_5


In [24]:
# rename condition
coords_df['condition'] = coords_df.apply(lambda row: ''.join([w[0] for w in row['condition'].split(' ')]), axis=1)

In [25]:
# rename level
coords_df['level'] = coords_df.level.apply(lambda l: ''.join(l.split('/')))

In [26]:
coords_df['cl'] = coords_df['condition'] + coords_df['level']

In [27]:
coords_df.condition.nunique()

5

In [28]:
coords_df.head(10)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl
0,4003253,702807833,8,SCS,L1L2,322.831858,227.964602,4003253_702807833,4003253_702807833_8,SCSL1L2
1,4003253,702807833,8,SCS,L2L3,320.571429,295.714286,4003253_702807833,4003253_702807833_8,SCSL2L3
2,4003253,702807833,8,SCS,L3L4,323.030303,371.818182,4003253_702807833,4003253_702807833_8,SCSL3L4
3,4003253,702807833,8,SCS,L4L5,335.292035,427.327434,4003253_702807833,4003253_702807833_8,SCSL4L5
4,4003253,702807833,8,SCS,L5S1,353.415929,483.964602,4003253_702807833,4003253_702807833_8,SCSL5S1
5,4003253,1054713880,4,RNFN,L4L5,187.961759,251.839388,4003253_1054713880,4003253_1054713880_4,RNFNL4L5
6,4003253,1054713880,4,RNFN,L5S1,198.240918,285.613767,4003253_1054713880,4003253_1054713880_4,RNFNL5S1
7,4003253,1054713880,5,RNFN,L3L4,187.227533,210.722753,4003253_1054713880,4003253_1054713880_5,RNFNL3L4
8,4003253,1054713880,6,RNFN,L1L2,194.569790,127.755258,4003253_1054713880,4003253_1054713880_6,RNFNL1L2
9,4003253,1054713880,6,RNFN,L2L3,191.632887,165.934990,4003253_1054713880,4003253_1054713880_6,RNFNL2L3


In [29]:
train_desc_df.series_description.unique()

array(['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2'], dtype=object)

In [30]:
train_desc_df.series_description.isna().sum()

0

In [31]:
coords_df.shape

(48692, 10)

In [32]:
coords_df = pd.merge(coords_df, train_desc_df.loc[:, ['ss_id', 'series_description']],  how='inner', left_on=['ss_id'], right_on=['ss_id'])

coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description
14244,1271819130,396937199,17,LSS,L2L3,356.814159,382.584071,1271819130_396937199,1271819130_396937199_17,LSSL2L3,Axial T2
10327,934012015,1947133002,6,RNFN,L1L2,182.789331,97.070524,934012015_1947133002,934012015_1947133002_6,RNFNL1L2,Sagittal T1


In [33]:
coords_df.shape

(48692, 11)

In [34]:
files_df.sample()

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription
29246,1780646606,3098929855,23,128,23,512,512,4.0,5.0,FFS,T2


In [35]:
# coords_df['plane'] = coords_df.apply(lambda row: train_desc_df[train_desc_df['ss_id'] == row['ss_id']].series_description.values[0], axis=1)

In [36]:
# coords_df.sample(5)

In [37]:
# check canal stenosis is noy only in axial plane
coords_df[(coords_df.condition == 'SCS') & (coords_df.series_description != 'Axial T2')].sample()

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description
45316,3985825195,2186866211,6,SCS,L3L4,320.0,268.707581,3985825195_2186866211,3985825195_2186866211_6,SCSL3L4,Sagittal T2/STIR


In [38]:
# get the positive slices
coords_df.groupby(['study_id','series_id']).instance.unique()

study_id    series_id 
4003253     702807833                              [8]
            1054713880               [4, 5, 6, 11, 12]
            2448190387          [3, 4, 11, 19, 28, 35]
4646740     3201256954    [15, 16, 22, 28, 29, 34, 40]
            3486248476           [5, 6, 7, 15, 16, 17]
                                      ...             
4287160193  1507070277                             [8]
            1820446240          [4, 9, 10, 16, 22, 28]
4290709089  3274612423                             [9]
            3390218084    [2, 3, 5, 6, 10, 15, 20, 21]
            4237840455                  [4, 5, 11, 12]
Name: instance, Length: 6291, dtype: object

In [39]:
coords_df.ss_id.nunique(), coords_df.instance_id.nunique()

(6291, 24546)

### Files_df

In [40]:
files_df = pd.read_csv(CFG.FILES_CSV)

In [41]:
files_df.sample(5)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription
136622,734370379,1655908293,10,1,10,512,512,5.0,6.0,HFS,T2
114580,3985825195,3514821988,8,1,8,512,512,4.0,5.0,HFS,T1
119746,4136088296,2455515522,43,-433,43,320,320,4.0,4.4,HFS,T2
21199,1557820205,1079852660,10,-2,10,384,384,4.0,4.4,HFS,T2
73240,2920127450,1211652526,2,91,2,384,384,4.0,4.4,HFS,NaN


In [42]:
# files_df.rename(columns={'patient': 'study_id', 'series': 'series_id', 'image': 'instance'}, inplace=True)

In [43]:
files_df.patientposition.value_counts(), files_df.patientposition.isna().sum()

(patientposition
 HFS    118249
 FFS     28969
 Name: count, dtype: int64,
 0)

In [44]:
files_df.groupby(['study_id']).patientposition.unique().value_counts()

patientposition
[HFS]    1585
[FFS]     390
Name: count, dtype: int64

In [45]:
files_df['ss_id'] = files_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)
files_df['instance_id'] = files_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}_{str(row["instancenumber"])}', axis=1)

In [46]:
source_dir = CFG.PNG_DIR
files_df['filename'] = files_df.apply(lambda row: f'{source_dir}\\{row.study_id}_{row.series_id}_{row.image}.png', axis=1)

In [47]:
files_df.sample()

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename
44541,2192457937,3771109646,14,0,14,320,320,4.0,5.6,HFS,T2,2192457937_3771109646,2192457937_3771109646_14,E:\data\RSNA2024\pngs_256\2192457937_377110964...


In [48]:
train_desc_df.sample()

,study_id,series_id,series_description,ss_id
2210,1529405332,4218128326,Axial T2,1529405332_4218128326


In [49]:
coords_df = pd.merge(coords_df, files_df[['instance_id', 'rows', 'columns', 'filename','patientposition']], left_on='instance_id', right_on='instance_id')

In [50]:
coords_df.shape

(48692, 15)

In [51]:
coords_df.sample()

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description,rows,columns,filename,patientposition
34386,3039901962,938004175,8,SCS,L2L3,384.685714,356.228571,3039901962_938004175,3039901962_938004175_8,SCSL2L3,Sagittal T2/STIR,768,768,E:\data\RSNA2024\pngs_256\3039901962_938004175...,FFS


In [52]:
# TODO: make sure we match coords corectly
coords_df['x_perc'] = coords_df['x'] / coords_df['columns']
coords_df['y_perc'] = coords_df['y'] / coords_df['rows']

In [53]:
ax, non_ax = coords_df[coords_df['series_description'] == 'Axial T2'], coords_df[coords_df['series_description'] != 'Axial T2']
ax.shape, non_ax.shape, coords_df.shape

((19220, 17), (29472, 17), (48692, 17))

In [54]:
coords_df.y.min()

2.063097514340344

In [55]:
for c in [ax, non_ax]:
    print(c['x_perc'].min(), c['y_perc'].min())
    print(c['x_perc'].max(), c['y_perc'].max())
    print('/////////////')
    # print(c['x_perc'].mean(), c['y_perc'].mean())

0.2849557522123894 0.3092621664050236
0.6631964653474212 0.8078096961632665
/////////////
0.0048828125 0.0040294873326959845
0.7684669901065448 0.9117647058823529
/////////////


In [56]:
files_df.shape

(147218, 14)

In [57]:
files_df = pd.merge(files_df, train_desc_df.loc[:, ['ss_id', 'series_description']],  how='inner', left_on=['ss_id'], right_on=['ss_id'])

files_df.sample(2)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename,series_description
25304,1676531728,3775504232,2,30,2,448,448,4.0,4.48,HFS,T2,1676531728_3775504232,1676531728_3775504232_2,E:\data\RSNA2024\pngs_256\1676531728_377550423...,Sagittal T2/STIR
115971,4031357862,271011448,4,63,4,384,384,4.0,5.20,HFS,NaN,4031357862_271011448,4031357862_271011448_4,E:\data\RSNA2024\pngs_256\4031357862_271011448...,Axial T2


In [58]:
files_df.shape

(147218, 15)

In [59]:
files_df['cl'] = 'H'
files_df['condition'] = 'H'

In [60]:
files_df.sample(5)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename,series_description,cl,condition
102685,3701224868,527018024,47,-545,47,640,640,3.5,3.5,HFS,T2,3701224868_527018024,3701224868_527018024_47,E:\data\RSNA2024\pngs_256\3701224868_527018024...,Axial T2,H,H
21797,1576224571,1449782109,29,-585,29,320,320,3.5,3.5,HFS,T2,1576224571_1449782109,1576224571_1449782109_29,E:\data\RSNA2024\pngs_256\1576224571_144978210...,Axial T2,H,H
81133,314041963,277371578,4,25,4,576,576,3.0,3.6,HFS,T2,314041963_277371578,314041963_277371578_4,E:\data\RSNA2024\pngs_256\314041963_277371578_...,Sagittal T2/STIR,H,H
34448,1906657742,4211307612,4,127,4,640,640,4.0,4.4,FFS,T2,1906657742_4211307612,1906657742_4211307612_4,E:\data\RSNA2024\pngs_256\1906657742_421130761...,Axial T2,H,H
129781,532925408,996418962,6,-74,6,640,640,4.0,4.8,HFS,T2,532925408_996418962,532925408_996418962_6,E:\data\RSNA2024\pngs_256\532925408_996418962_...,Axial T2,H,H


In [61]:
files_df['inst_min'] = files_df.groupby('ss_id')['instancenumber'].transform('min')
files_df['inst_max'] = files_df.groupby('ss_id')['instancenumber'].transform('max')

files_df['inst'] = files_df['instancenumber'] - files_df['inst_min']
files_df['inst_perc'] = files_df['inst'] / files_df['inst_max']

files_df.head(2)

,study_id,series_id,image,proj,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,...,ss_id,instance_id,filename,series_description,cl,condition,inst_min,inst_max,inst,inst_perc
0,100206310,1012284084,1,-394,1,320,320,3.5,3.5,HFS,...,100206310_1012284084,100206310_1012284084_1,E:\data\RSNA2024\pngs_256\100206310_1012284084...,Axial T2,H,H,1,60,0,0.00
1,100206310,1012284084,10,-427,10,320,320,3.5,3.5,HFS,...,100206310_1012284084,100206310_1012284084_10,E:\data\RSNA2024\pngs_256\100206310_1012284084...,Axial T2,H,H,1,60,9,0.15


In [66]:
coords_df = pd.merge(coords_df, files_df[['instance_id','inst_perc']], left_on='instance_id', right_on='instance_id')

coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,series_description,rows,columns,filename,patientposition,x_perc,y_perc,inst_perc
21850,1972129014,3203550406,5,RNFN,L4L5,120.689564,156.237781,1972129014_3203550406,1972129014_3203550406_5,RNFNL4L5,Sagittal T1,256,256,E:\data\RSNA2024\pngs_256\1972129014_320355040...,HFS,0.471444,0.610304,0.222222
42428,3736664503,91239887,10,SCS,L3L4,243.961538,241.934066,3736664503_91239887,3736664503_91239887_10,SCSL3L4,Sagittal T2/STIR,448,448,E:\data\RSNA2024\pngs_256\3736664503_91239887_...,HFS,0.544557,0.540031,0.500000


### Save results

In [67]:
train_df.to_csv(CFG.DEST_FOLDER / 'train.csv', index=False)

In [68]:
coords_df.to_csv(CFG.DEST_FOLDER / 'train_label_coordinates.csv', index=False)

In [69]:
files_df.to_csv(CFG.DEST_FOLDER / 'train_files.csv', index=False)